In [1]:
import os
import pandas as pd
from Bio import SeqIO

# Load data
aggregated_tests = pd.read_excel('/Users/hanli/Desktop/FYP/PAML/aggregated_all_tests_combined.xlsx', sheet_name='remove_0_rows')
final_matrix = pd.read_excel('/Users/hanli/Desktop/FYP/PAML/final_matrix_with_metadata.xlsx', sheet_name='metadata')
genes_dir = '/Users/hanli/Desktop/FYP/PAML/genomes/kofamscan/csv'
gff_dir = '/Users/hanli/Desktop/FYP/PAML/genomes/gff'
fna_dir = '/Users/hanli/Desktop/FYP/PAML/genomes/fna'
faa_dir = '/Users/hanli/Desktop/FYP/PAML/genomes/faa'
ffn_dir = '/Users/hanli/Desktop/FYP/PAML/genomes/ffn'


def extract_sequences(genome, gene, target_id):
    ffn_file = os.path.join(ffn_dir, f'{genome}.ffn')
    try:
        with open(ffn_file) as ffn:
            for record in SeqIO.parse(ffn_file, 'fasta'):
                if target_id in record.id:
                    return record
            return None
    except FileNotFoundError:
        print(f"File not found: {ffn_file}")
        return


def extract_aa_sequences(genome, gene, target_id):
    faa_file = os.path.join(faa_dir, f'{genome}.faa')
    try:
        with open(faa_file) as faa:
            for record in SeqIO.parse(faa_file, 'fasta'):
                if target_id in record.id:
                    return record
            return None
    except FileNotFoundError:
        print(f"File not found: {faa_file}")
        return

for _, row in aggregated_tests.iterrows():
    if row['method'] == 'KOfamScan' and row['Order'] in ['Rhizobiales', 'Pseudomonadales'] and row['selected'] == 1:
        gene = row['gene']
        niche = row['niche']
        order = row['Order']
        niche_new = niche.replace(' & ', '_')
        print("***********")
        print(gene)
        print(niche)
        print(order)
        print("***********")
        # Filter genomes
        if niche == "rhizo & soil":
            spheres = ["Rhizosphere", "Soil"]
        elif niche == "phyllo & soil":
            spheres = ["Phyllosphere", "Soil"]
        else:
            continue
            
        selected_genomes = final_matrix[(final_matrix['sphere'].isin(spheres)) & (final_matrix['Order'] == order)]
        
        # Extract sequences
        sequences = []
        aa_sequences = []
        for genome in selected_genomes['Gene']:
            csv_file = os.path.join(genes_dir, f'{genome}.csv')
            with open(csv_file) as csvfile:
                for line in csvfile:
                    if gene in line:
                        identifier, _ = line.strip().split(',', 1)
                        scaffold_id, cds_number = identifier.rsplit('_', 1)
                        scaffold_id = scaffold_id.replace('"', '')
                        cds_number = cds_number.replace('"', '')

                        # Extract the corresponding target ID from the GFF file
                        gff_file = os.path.join(gff_dir, f'{genome}.gff')
                        
                        # Define a custom exception for breaking out of nested loops
                        class TargetIdFound(Exception):
                            pass
                        
                        target_id = None  # Initialize target_id with a default value

                        try:
                            with open(gff_file) as gff:
                                cds_list = []
                                for line in gff:
                                    if line.startswith(scaffold_id) and "CDS" in line:
                                        cds_list.append(line)
                                cds_number_int = int(cds_number)
                                cds_list = [cds for cds in cds_list if "CDS" in cds]
                                if cds_number_int > len(cds_list):
                                    raise TargetIdFound()
                                target_cds = cds_list[cds_number_int - 1]
                                target_id = target_cds.split("ID=")[1].split(";")[0]
                                raise TargetIdFound()
                        except TargetIdFound:
                            pass

                        # Proceed only if target_id is defined
                        if target_id is not None:
                            seq = extract_sequences(genome, gene, target_id)
                            aa_seq = extract_aa_sequences(genome, gene, target_id)
                            if seq is not None:
                                sequences.append((genome, seq))
                            if aa_seq is not None:
                                aa_sequences.append((genome, aa_seq))
                                # ... The previous code

                                # Save nucleotide sequences to a FASTA file
                                niche_order = f"{niche.replace(' & ', '_')}_{order}"
                                nucleotide_output_file_path = f'/Users/hanli/Desktop/FYP/PAML/new_fna_faa/{gene}_{niche_order}_ffn_output.fasta'
                                with open(nucleotide_output_file_path, 'w') as fasta_file:
                                    for genome, record in sequences:
                                        header = f"{genome}_{record.id} {record.description.split(' ', 1)[1]}"
                                        fasta_file.write(f'>{header}\n{record.seq}\n')

                                # Save amino acid sequences to a FASTA file
                                aa_output_file_path = f'/Users/hanli/Desktop/FYP/PAML/new_fna_faa/{gene}_{niche_order}_faa_output.fasta'
                                with open(aa_output_file_path, 'w') as fasta_file:
                                    for genome, record in aa_sequences:
                                        header = f"{genome}_{record.id} {record.description.split(' ', 1)[1]}"
                                        fasta_file.write(f'>{header}\n{record.seq}\n')

***********
K00548
rhizo & soil
Rhizobiales
***********
***********
K00651
rhizo & soil
Rhizobiales
***********
***********
K00799
rhizo & soil
Pseudomonadales
***********
***********
K01505
rhizo & soil
Rhizobiales
***********
***********
K07160
phyllo & soil
Rhizobiales
***********
***********
K15554
rhizo & soil
Pseudomonadales
***********
***********
K22955
rhizo & soil
Rhizobiales
***********
***********
K00003
rhizo & soil
Pseudomonadales
***********
***********
K00032
rhizo & soil
Pseudomonadales
***********
***********
K00036
rhizo & soil
Pseudomonadales
***********
***********
K00383
rhizo & soil
Pseudomonadales
***********
***********
K02439
phyllo & soil
Pseudomonadales
***********
***********
K00387
phyllo & soil
Rhizobiales
***********
***********
K00432
phyllo & soil
Rhizobiales
***********
***********
K00548
phyllo & soil
Rhizobiales
***********
***********
K00640
phyllo & soil
Rhizobiales
***********
***********
K05301
phyllo & soil
Rhizobiales
***********
***********
K